In [1]:
import pandas as pd
import numpy as np

In [49]:
INPUT_PATH = '../input/'
OUTPUT_PATH = '../output/'

users = pd.read_csv(INPUT_PATH + 't_user.csv')
orders = pd.read_csv(INPUT_PATH + 't_order.csv')
loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

loans.loan_amount = 5**loans.loan_amount
loans_sum.loan_sum  = 5 ** loans_sum.loan_sum

In [75]:
#watch 34939
uids = ['4']
print(clicks.loc[clicks['uid'].isin(uids)])



          uid           click_time  pid  param                Date  \
140128      4  2016-11-23 16:03:43   10     16 2016-11-23 16:03:43   
153789      4  2016-08-29 00:49:49    9      1 2016-08-29 00:49:49   
168961      4  2016-08-03 15:25:32    9      2 2016-08-03 15:25:32   
168962      4  2016-08-03 16:25:20    6      5 2016-08-03 16:25:20   
168963      4  2016-08-03 15:25:37    9      1 2016-08-03 15:25:37   
168964      4  2016-08-03 15:25:30    9      1 2016-08-03 15:25:30   
168965      4  2016-08-03 15:26:01    9      3 2016-08-03 15:26:01   
168966      4  2016-08-03 15:26:33    8      1 2016-08-03 15:26:33   
168967      4  2016-08-03 15:27:38   10     16 2016-08-03 15:27:38   
168968      4  2016-08-03 15:27:55   10     16 2016-08-03 15:27:55   
168969      4  2016-08-03 15:26:29    8      1 2016-08-03 15:26:29   
168970      4  2016-08-03 15:25:35    9      4 2016-08-03 15:25:35   
291464      4  2016-08-08 17:45:15    8      1 2016-08-08 17:45:15   
291465      4  2016-

In [76]:
print(orders.loc[orders['uid'].isin(uids)])

         uid    buy_time     price  qty  cate_id  discount
37202      4  2016-08-26  3.520749    1        3  3.321160
614023     4  2016-10-04  2.861353    1       13  0.000000
643192     4  2016-09-13  5.338581    1       33  0.000000
695501     4  2016-10-04  2.665812    1       13  0.000000
791687     4  2016-09-13  2.855108    1       33  0.000000
966981     4  2016-08-27  2.974636    1       22  2.393552
1144396    4  2016-08-13  3.478495    1       22  2.133656
1324907    4  2016-08-13  1.358270    1       15  0.000000
1645012    4  2016-09-13  2.855108    1       33  0.000000
2671252    4  2016-11-28  2.866304    1       26  0.000000
3124949    4  2016-09-20  2.436859    1       26  0.000000
3971568    4  2016-10-04  3.949272    1       13  0.000000
4237477    4  2016-09-13  2.855108    1       33  0.000000
4947185    4  2016-09-12  3.256881    1       38  1.891668
5097317    4  2016-08-13  3.430677    1       22  2.730425
5226430    4  2016-08-13  4.781926    1       22  3.9022

In [77]:

print(loans.loc[loans['uid'].isin(uids)].sort_values('loan_time', ascending=False))

        uid            loan_time  loan_amount  plannum                Date  \
167918    4  2016-11-27 14:10:18       2001.0        3 2016-11-27 14:10:18   
160681    4  2016-10-29 17:52:58      12001.0       12 2016-10-29 17:52:58   
190411    4  2016-10-14 19:09:13       4001.0        1 2016-10-14 19:09:13   
70516     4  2016-08-12 21:51:25       4001.0        1 2016-08-12 21:51:25   
34423     4  2016-08-05 08:11:38       5001.0        1 2016-08-05 08:11:38   

        transaction_month  
167918                 11  
160681                 10  
190411                 10  
70516                   8  
34423                   8  


In [78]:
print(loans_sum.loc[loans_sum['uid'].isin(uids)])

      uid    month  loan_sum
4862    4  2016-11    2001.0


In [79]:
print(users.loc[users['uid'].isin(uids)])

       uid  age  sex active_date     limit
78039    4   30    1  2016-03-26  5.292154


In [64]:
loans['Date'] = pd.to_datetime(loans['loan_time'], errors='coerce')
loans['transaction_month'] = loans['Date'].dt.month

In [85]:
loan_of_nov = loans.loc[loans.transaction_month==11].groupby(by='uid', as_index=False).sum()

loan_of_nov.merge(loans_sum, how='left', on='uid')

# comp = loan_of_nov.merge(loans_sum, how='left', on='uid')
    # Compute smoothing

,uid,loan_amount,plannum,transaction_month,month,loan_sum
0,4,2001.000000,3,11,2016-11,2001.000000
1,5,40000.999999,12,11,2016-11,40000.999999
2,6,24000.999998,6,11,2016-11,24000.999998
3,17,9001.000000,12,11,2016-11,9001.000000
4,19,13000.999999,12,11,2016-11,13000.999999
5,23,501.000000,1,11,2016-11,501.000000
6,39,6607.000000,84,77,2016-11,6601.000000
7,43,5001.000000,1,11,2016-11,5001.000000
8,45,25003.000001,3,33,2016-11,25000.999999
9,56,50001.000003,1,11,2016-11,50001.000003
